</center><img src="coverage_workflow_image.png"  width="1000" height="300"/></center>

In [1]:
# import required python libraries
#import geopandas as gpd  # for geo-data structure
import numpy as np  # for some operation better done with numpy
import matplotlib.pyplot as plt  # for creating plots
#import shapely.geometry  # for geometry data crunching
#from shapely.geometry import Point  # specifically for point geometry data manipulation
#import keplergl
#from keplergl import KeplerGl
import datacompy
import pandas as pd
crs = {'init': 'epsg:4326'}
#from shapely.geometry import Point
import mpl_toolkits


In [11]:
# bring in the 3 to 4 different coverage dataset from FB, GSMA, ITU and Ericsson as well as the master school_geolocation

In [2]:
d_f = pd.read_csv('./data/Brazil_Connectivity.csv',low_memory=False,encoding='utf-8')
d_i = pd.read_csv('./data/BRA_coverage_ITU.csv',low_memory=False,encoding='utf-8')
d_e = pd.read_csv('./data/Brazil_Ericsson_v1.csv',low_memory=False,encoding='utf-8')
#d_g = pd.read_csv('./data/GSMA_RW_Schools_mobile_coverage_v1.csv',low_memory=False,encoding='utf-8')
s_master = pd.read_csv('./data/BR_school_geolocation_master.csv',low_memory=False,encoding='utf-8')

In [6]:
d_f.shape

(175704, 28)

In [7]:
d_e.head(2)

,uuid,country,source,giga_school_id,SCHOOL_CODE,SCHOOL_NAME,LATITUDE,LONGITUDE,hex8,dq_score,...,2G_Best_Coverage_provider,3G_Best_Coverage_provider,4G_Best_Coverage_provider,5G_Best_Coverage_provider,2G_coverage_present,3G_coverage_present,4G_coverage_present,5G_coverage_present,Distance_to_nearest_tower,Data_Source
0,82679ff0-f358-33d0-b3ce-4fc4c2740b7e,Brazil,government,G_BRA_000001,11000023,EEEE ABNAEL MACHADO DE LIMA - CENE,-8.7585,-63.8540,888a05915dfffff,0.94,...,Vivo,Claro,Claro,None,Yes,Yes,Yes,No,-1,Ericsson
1,115d27ba-82a1-3744-a3fb-90f216f54f6f,Brazil,government,G_BRA_000002,11000040,EMEIEF PEQUENOS TALENTOS,-8.7937,-63.8839,888a059319fffff,1.00,...,Vivo,Claro,Claro,None,Yes,Yes,Yes,No,-1,Ericsson


In [10]:
d_e.columns

Index(['uuid', 'country', 'source', 'giga_school_id', 'SCHOOL_CODE',
       'SCHOOL_NAME', 'LATITUDE', 'LONGITUDE', 'hex8', 'dq_score', 'duplicate',
       'source_timestamp', 'master_timestamp', 'school_archive',
       'district_code', 'student_count', 'computer_availability',
       'school_type', 'school_region', 'internet_availability',
       'internet_speed_Mbps', 'Coverage', 'Coverage_type', '2G_coverage',
       '3G_coverage', '4G_coverage', '5G_coverage', '2G_coverage_distance',
       '3G_coverage_distance', '4G_coverage_distance', '5G_coverage_distance',
       '2G_Best_Coverage_provider', '3G_Best_Coverage_provider',
       '4G_Best_Coverage_provider', '5G_Best_Coverage_provider',
       '2G_coverage_present', '3G_coverage_present', '4G_coverage_present',
       '5G_coverage_present', 'Distance_to_nearest_tower', 'Data_Source'],
      dtype='object')

In [11]:
d_e.rename(columns={'SCHOOL_NAME':'school_name', '2G_coverage':'2G_quality',
                    '3G_coverage':'3G_quality','4G_coverage':'4G_quality','2G_coverage_present':'2G_coverage', 
                    '3G_coverage_present':'3G_coverage','4G_coverage_present':'4G_coverage',
                    'Coverage':'coverage', 'Coverage_type':'coverage_type'}, inplace=True)
#d_i.rename(columns={'giga_schoo': 'giga_school_id','school_nam':'school_name'}, inplace=True)
#d_f.rename(columns={'school_id': 'giga_school_id','name':'school_name'}, inplace=True)

In [56]:
# cross-profiles the sources with master data for completeness and correctness

In [44]:
# create a separate dfs
master = s_master.copy()
fb = d_f.copy()
ec = d_e.copy()
itu = d_i.copy()
#gsma = d_g.copy()
# for facebook
cross_profile_fb_master = datacompy.Compare(
master,
fb,
join_columns='giga_school_id',  
df1_name='Master', 
df2_name='Facebook'
)
#for ericsson
cross_profile_e_master = datacompy.Compare(
master,
ec,
join_columns='giga_school_id',  
df1_name='Master', 
df2_name='Ericsson'
)
# for ITU
cross_profile_itu_master = datacompy.Compare(
master,
itu,
join_columns='giga_school_id',  
df1_name='Master', 
df2_name='ITU'
)
print(cross_profile_fb_master.report())
print(cross_profile_e_master.report())
print(cross_profile_itu_master.report())
#print(cross_profile_gsma_master.report())

#print(cross_profile_fb_master.report())
print(cross_profile_fb_master.report())
fm = open("./data/cross_profile_fb_master_br_report.txt", "a")
print(cross_profile_fb_master.report(), file=fm)
fm.close()

#print(cross_profile_Ericsson_master.report())
print(cross_profile_e_master.report())
em = open("./data/cross_profile_e_master_br_report.txt", "a")
print(cross_profile_e_master.report(), file=em)
em.close()

#print(cross_profile_itu_master.report())
print(cross_profile_itu_master.report())
im = open("./data/cross_profile_itu_master_br_report.txt", "a")
print(cross_profile_itu_master.report(), file=im)
im.close()

DataComPy Comparison
--------------------

DataFrame Summary
-----------------

  DataFrame  Columns    Rows
0    Master       21  175704
1  Facebook       33  175704

Column Summary
--------------

Number of columns in common: 17
Number of columns in Master but not in Facebook: 4
Number of columns in Facebook but not in Master: 16

Row Summary
-----------

Matched on: giga_school_id
Any duplicates on match values: No
Absolute Tolerance: 0
Relative Tolerance: 0
Number of rows in common: 175,704
Number of rows in Master but not in Facebook: 0
Number of rows in Facebook but not in Master: 0

Number of rows with some compared columns unequal: 175,704
Number of rows with all compared columns equal: 0

Column Comparison
-----------------

Number of columns compared with some values unequal: 1
Number of columns compared with all values equal: 16
Total number of values which compare unequal: 175,704

Columns with Unequal Values or Types
------------------------------------

      Column Maste

In [58]:
# some colunm names standardization

In [59]:
#d_e.rename(columns={'SCHOOL_CODE':'school_id', 'SCHOOL_NAME':'school_name'}, inplace=True)
#d_i.rename(columns={'giga_schoo': 'giga_school_id','source_sch':'school_id', 'name':'school_name'}, inplace=True)
#d_g.rename(columns={'giga_schoo': 'giga_school_id'}, inplace=True)

In [60]:
# cross-profiles the sources with master data for completeness and correctness

In [61]:
# they usually don't come with "coverage" colunms, we create it at source level using the following codes

In [13]:
#d_e['coverage'] = np.where((d_e['2G_coverage']== 'None') & (d_e['3G_coverage']== 'None')& (d_e['4G_coverage']== 'None'), 'NO', 'YES')
d_f['coverage'] = np.where((d_f['percent_2g'].isnull()) & (d_f['percent_3g'].isnull())& (d_f['percent_4g'].isnull()), 'NO', 'YES')
d_f['coverage'] = np.where((d_f['percent_2g']<=0) & (d_f['percent_3g']<=0) & (d_f['percent_4g']<=0), 'NO', 'YES')
d_i['coverage'] = np.where((d_i['2G'].isnull()) & (d_i['3G'].isnull())& (d_i['4G'].isnull()), 'NO', 'YES')
d_i['coverage'] = np.where((d_i['2G']<=0) & (d_i['3G']<=0) & (d_i['4G']<=0), 'NO', 'YES')


In [63]:
# create the 2G, 3G and 4G coverage columns with binary values at source level using the following codes

In [14]:
d_f['2G_coverage'] = np.where((d_f['percent_2g'] > 0), 'YES', 'NO')
d_f['3G_coverage'] = np.where((d_f['percent_3g'] > 0), 'YES', 'NO')
d_f['4G_coverage'] = np.where((d_f['percent_4g'] > 0), 'YES', 'NO')

In [15]:
d_i['2G_coverage'] = np.where((d_i['2G'] >=1), 'YES', 'NO')
d_i['3G_coverage'] = np.where((d_i['3G'] == 1), 'YES', 'NO')
d_i['4G_coverage'] = np.where((d_i['4G'] ==1), 'YES', 'NO')

In [66]:
#d_e.rename(columns={'2G_coverage':'2G_coverage_quality', '3G_coverage':'3G_coverage_quality', '4G_coverage': '4G_coverage_quality'}, inplace=True)

In [67]:
# they usually don't come with "coverage_type" colunms, we create it at source level using the following function and codes

In [16]:
def coverage_type(df):
    if df['4G_coverage'] == 'YES':
        return "4G"
    elif df['3G_coverage'] == 'YES':
        return "3G"
    elif df['2G_coverage'] == 'YES':
        return "2G"
    else:
        return "None"

In [17]:
d_f['coverage_type'] = d_f.apply((lambda x: coverage_type(x)),axis=1)
#d_e['coverage_type'] = d_e.apply((lambda x: coverage_type(x)),axis=1)
d_i['coverage_type'] = d_i.apply((lambda x: coverage_type(x)),axis=1)


In [18]:
d_e.head(1)

,uuid,country,source,giga_school_id,SCHOOL_CODE,school_name,LATITUDE,LONGITUDE,hex8,dq_score,...,2G_Best_Coverage_provider,3G_Best_Coverage_provider,4G_Best_Coverage_provider,5G_Best_Coverage_provider,2G_coverage,3G_coverage,4G_coverage,5G_coverage_present,Distance_to_nearest_tower,Data_Source
0,82679ff0-f358-33d0-b3ce-4fc4c2740b7e,Brazil,government,G_BRA_000001,11000023,EEEE ABNAEL MACHADO DE LIMA - CENE,-8.7585,-63.854,888a05915dfffff,0.94,...,Vivo,Claro,Claro,None,Yes,Yes,Yes,No,-1,Ericsson


In [71]:
#d_e.rename(columns={'2G_coverage_present':'2G_coverage', '3G_coverage_present':'3G_coverage','4G_coverage_present':'4G_coverage', 'Coverage':'coverage', 'Coverage_type':'coverage_type'}, inplace=True)

In [72]:
# isolating the needed colunms from the rest

In [19]:
features = ['giga_school_id','coverage','coverage_type','2G_coverage','3G_coverage','4G_coverage'] 

In [20]:
d_fc = d_f[features]
#d_ec = d_e[features]
d_it = d_i[features]
d_ec = d_e[features]

In [31]:
# more sanity checks for nan on giga_school_id

In [21]:
d_fc = d_fc[d_fc['giga_school_id'].notna()]
#d_ec = d_ec[d_ec['giga_school_id'].notna()]
d_it = d_it[d_it['giga_school_id'].notna()]
d_ec = d_ec[d_ec['giga_school_id'].notna()]

In [22]:
d_ec.loc[d_ec['2G_coverage'] == 'Yes', '2G_coverage'] = 'YES'
d_ec.loc[d_ec['3G_coverage'] == 'Yes', '3G_coverage'] = 'YES'
d_ec.loc[d_ec['4G_coverage'] == 'Yes', '4G_coverage'] = 'YES'
d_ec.loc[d_ec['coverage'] == 'Yes', 'coverage'] = 'YES'

d_ec.loc[d_ec['2G_coverage'] == 'No', '2G_coverage'] = 'NO'
d_ec.loc[d_ec['3G_coverage'] == 'No', '3G_coverage'] = 'NO'
d_ec.loc[d_ec['4G_coverage'] == 'No', '4G_coverage'] = 'NO'
d_ec.loc[d_ec['coverage'] == 'No', 'coverage'] = 'NO'


In [23]:
d_ec.loc[d_ec['2G_coverage'].isnull(), '2G_coverage'] = 'NO'
d_ec.loc[d_ec['3G_coverage'].isnull(), '3G_coverage'] = 'NO'
d_ec.loc[d_ec['4G_coverage'].isnull(), '4G_coverage'] = 'NO'
d_ec.loc[d_ec['coverage'].isnull(), 'coverage'] = 'NO'
d_ec.loc[d_ec['coverage_type'].isnull(), 'coverage_type'] = 'NO'

In [76]:
# visually comparing the sources for matches and mismatches across the columns

In [26]:
d_fc.shape

(175704, 6)

In [27]:
df_all = pd.concat([d_fc.set_index('giga_school_id'), d_ec.set_index('giga_school_id'),d_it.set_index('giga_school_id')], 
                   axis='columns', keys=['Facebook', 'Ericsson', 'ITU'])
df_all.head()

Facebook                                                    \
             coverage coverage_type 2G_coverage 3G_coverage 4G_coverage   
G_BRA_000316      YES            4G         YES         YES         YES   
G_BRA_000168      YES            4G         YES         YES         YES   
G_BRA_000576      YES            4G         YES         YES         YES   
G_BRA_000627      YES            4G         YES         YES         YES   
G_BRA_000711      YES            4G         YES         YES         YES   

             Ericsson                                                    \
             coverage coverage_type 2G_coverage 3G_coverage 4G_coverage   
G_BRA_000316      YES            4G         YES         YES         YES   
G_BRA_000168      YES            4G         YES         YES         YES   
G_BRA_000576      YES            4G         YES         YES         YES   
G_BRA_000627       NO          None          NO          NO          NO   
G_BRA_000711       NO          None          NO          NO          NO   

                  ITU                                                    
             coverage coverage_type 2G_coverage 3G_coverage 4G_coverage  
G_BRA_000316      YES            3G         YES         YES          NO  
G_BRA_000168      YES            3G         YES         YES          NO  
G_BRA_000576      YES            3G         YES         YES          NO  
G_BRA_000627       NO          None          NO          NO          NO  
G_BRA_000711       NO          None          NO          NO          NO

In [28]:
d_ec.head()

,giga_school_id,coverage,coverage_type,2G_coverage,3G_coverage,4G_coverage
0,G_BRA_000001,YES,4G,YES,YES,YES
1,G_BRA_000002,YES,4G,YES,YES,YES
2,G_BRA_000003,YES,4G,YES,YES,YES
3,G_BRA_000004,YES,4G,YES,YES,YES
4,G_BRA_000005,YES,4G,YES,YES,YES


In [29]:
df_final = df_all.swaplevel(axis='columns')[d_fc.columns[1:]]
df_final.head()

coverage               coverage_type                2G_coverage  \
             Facebook Ericsson  ITU      Facebook Ericsson   ITU    Facebook   
G_BRA_000316      YES      YES  YES            4G       4G    3G         YES   
G_BRA_000168      YES      YES  YES            4G       4G    3G         YES   
G_BRA_000576      YES      YES  YES            4G       4G    3G         YES   
G_BRA_000627      YES       NO   NO            4G     None  None         YES   
G_BRA_000711      YES       NO   NO            4G     None  None         YES   

                           3G_coverage               4G_coverage               
             Ericsson  ITU    Facebook Ericsson  ITU    Facebook Ericsson ITU  
G_BRA_000316      YES  YES         YES      YES  YES         YES      YES  NO  
G_BRA_000168      YES  YES         YES      YES  YES         YES      YES  NO  
G_BRA_000576      YES  YES         YES      YES  YES         YES      YES  NO  
G_BRA_000627       NO   NO         YES       NO   NO         YES       NO  NO  
G_BRA_000711       NO   NO         YES       NO   NO         YES       NO  NO

In [30]:
def highlight_diff(data, color='lightcoral'):
    attr = 'background-color: {}'.format(color)
    other = data.xs('Facebook', axis='columns', level=-1)
    return pd.DataFrame(np.where(data.ne(other, level=0), attr, ''),
                        index=data.index, columns=data.columns)

#df_final.style.apply(highlight_diff, axis=None)

In [92]:
# coverage comparison report

In [43]:
# create a separate dfs
fbc = d_fc.copy()
#ecc = d_ec.copy()
ituc = d_it.copy()
ecc = d_ec.copy()

# Facebook&ITU
cross_profile_fb_itu = datacompy.Compare(
fbc,
ituc,
join_columns='giga_school_id',  
df1_name='Facebook', 
df2_name='ITU'
)

cross_profile_facebook_ec = datacompy.Compare(
fbc,
ecc,
join_columns='giga_school_id',  
df1_name='Facebook', 
df2_name='GSMA'
)


#print(cross_profile_fb_ec.report())
print(cross_profile_fb_itu.report())
fi = open("./data/cross_profile_fb_itu_report.txt", "a")
print(cross_profile_fb_itu.report(), file=fi)
fi.close()
#print(cross_profile_Ericsson_itu.report())
print(cross_profile_facebook_ec.report())
fe = open("./data/cross_profile_fb_ec_report.txt", "a")
print(cross_profile_fb_ec.report(), file=fe)
fe.close()

DataComPy Comparison
--------------------

DataFrame Summary
-----------------

  DataFrame  Columns    Rows
0  Facebook        6  175704
1       ITU        6  175704

Column Summary
--------------

Number of columns in common: 6
Number of columns in Facebook but not in ITU: 0
Number of columns in ITU but not in Facebook: 0

Row Summary
-----------

Matched on: giga_school_id
Any duplicates on match values: No
Absolute Tolerance: 0
Relative Tolerance: 0
Number of rows in common: 175,704
Number of rows in Facebook but not in ITU: 0
Number of rows in ITU but not in Facebook: 0

Number of rows with some compared columns unequal: 152,254
Number of rows with all compared columns equal: 23,450

Column Comparison
-----------------

Number of columns compared with some values unequal: 5
Number of columns compared with all values equal: 1
Total number of values which compare unequal: 486,306

Columns with Unequal Values or Types
------------------------------------

          Column Facebook dt

NameError: name 'cross_profile_fb_ec' is not defined

In [39]:
# merging and selecting the most frequent values across the sources on each row and colunm
# creating a master coverage dataset and merging with the master school location dataset

In [32]:
#import statistics as stats
df_c = pd.concat([d_fc,d_ec,d_it])
coverage = df_c.groupby(['giga_school_id'],as_index=False).agg(lambda x:x.value_counts().index[0])
coverage['coverage_type'] = coverage.apply((lambda x: coverage_type(x)),axis=1)
#coverage = df_c.groupby(['giga_school_id'],as_index=False).agg(lambda x: stats.mode(x)[0][0])


In [33]:
coverage.head()

,giga_school_id,coverage,coverage_type,2G_coverage,3G_coverage,4G_coverage
0,G_BRA_000001,YES,4G,YES,YES,YES
1,G_BRA_000002,YES,4G,YES,YES,YES
2,G_BRA_000003,YES,4G,YES,YES,YES
3,G_BRA_000004,YES,4G,YES,YES,YES
4,G_BRA_000005,YES,4G,YES,YES,YES


In [34]:
coverage['coverage'] = np.where((coverage['2G_coverage']=='YES') | (coverage['3G_coverage']=='YES') | (coverage['4G_coverage']=='YES'), 'YES', 'NO')

In [35]:
coverage[coverage['coverage']=='NO'].head(10)

,giga_school_id,coverage,coverage_type,2G_coverage,3G_coverage,4G_coverage
10,G_BRA_000011,NO,None,NO,NO,NO
16,G_BRA_000017,NO,None,NO,NO,NO
17,G_BRA_000018,NO,None,NO,NO,NO
19,G_BRA_000020,NO,None,NO,NO,NO
24,G_BRA_000025,NO,None,NO,NO,NO
27,G_BRA_000028,NO,None,NO,NO,NO
28,G_BRA_000029,NO,None,NO,NO,NO
30,G_BRA_000031,NO,None,NO,NO,NO
34,G_BRA_000035,NO,None,NO,NO,NO
37,G_BRA_000038,NO,None,NO,NO,NO


In [36]:
coverage[coverage['coverage']=='NO'].shape

(23888, 6)

In [97]:
#coverage.loc[coverage['coverage_type'] == 'None', 'coverage_type'] = d_gs.coverage_type

In [37]:
master.head()

,uuid,country,source,giga_school_id,school_id,name,lat,lon,hex8,dq_score,...,source_timestamp,master_timestamp,school_archive,district_code,student_count,computer_availability,school_type,school_region,connectivity,internet_speed_mbps
0,82679ff0-f358-33d0-b3ce-4fc4c2740b7e,Brazil,government,G_BRA_000001,11000023,EEEE ABNAEL MACHADO DE LIMA - CENE,-8.7585,-63.8540,888a05915dfffff,0.94,...,2021-04,32:36.9,No,110020505,174,Yes,Estadual,Urban,Yes,5.0
1,115d27ba-82a1-3744-a3fb-90f216f54f6f,Brazil,government,G_BRA_000002,11000040,EMEIEF PEQUENOS TALENTOS,-8.7937,-63.8839,888a059319fffff,1.00,...,2021-04,32:36.9,No,110020505,205,No,Municipal,Urban,Yes,5.0
2,3d7fb661-61a4-33b3-bc3a-33455b75f7d9,Brazil,government,G_BRA_000003,11000201,EMEF PROF HERBERT DE ALENCAR,-8.7640,-63.8783,888a059061fffff,0.94,...,2021-04,32:36.9,No,110020505,407,Yes,Municipal,Urban,Yes,5.0
3,9507b9e6-4ea6-3cfc-b0c5-4c0b9f64c711,Brazil,government,G_BRA_000004,11000260,COLEGIO TIRADENTES DA POLICIA MILITAR DO ESTAD...,-8.7383,-63.8791,888a059021fffff,1.00,...,2021-04,32:36.9,No,110020505,1457,Yes,Estadual,Urban,Yes,5.0
4,b5ec97ef-a418-35da-b161-3971c87ac36d,Brazil,government,G_BRA_000005,11000309,ESCOLA MUNICIPAL DE EDUCACAO INFANTIL MARISE C...,-8.7508,-63.9087,888a05900bfffff,1.00,...,2021-04,32:36.9,No,110020505,499,No,Municipal,Urban,No,0.0


In [38]:
master_w_coverage = master.merge(coverage, how="left", on=["giga_school_id"])
master_w_coverage.to_csv('./data/BR_master_w_coverage.csv', encoding='utf-8')
master_w_coverage_connectivity = master_w_coverage.copy()
#master_w_coverage_connectivity['connectivity_availability'] = np.where((master_w_coverage_connectivity['connectivity_type'] != 'None'), 'YES', 'NO')
master_w_coverage_connectivity.loc[master_w_coverage_connectivity['coverage'] == 'NO', 'connectivity'] = 'NO'
master_w_coverage_connectivity.loc[master_w_coverage_connectivity['coverage'] == 'NO', 'coverage_type'] = 'None'
master_w_coverage_connectivity.loc[master_w_coverage_connectivity['coverage_type'] == 'None', 'coverage'] = 'NO'
master_w_coverage_connectivity.loc[master_w_coverage_connectivity['coverage_type'] == 'None', 'coverage_type'] = 'no coverage'
master_w_coverage_connectivity.to_csv('./data/BR_master_w_coverage_connectivity.csv', encoding='utf-8')

In [39]:
master_w_coverage.head(2)

,uuid,country,source,giga_school_id,school_id,name,lat,lon,hex8,dq_score,...,computer_availability,school_type,school_region,connectivity,internet_speed_mbps,coverage,coverage_type,2G_coverage,3G_coverage,4G_coverage
0,82679ff0-f358-33d0-b3ce-4fc4c2740b7e,Brazil,government,G_BRA_000001,11000023,EEEE ABNAEL MACHADO DE LIMA - CENE,-8.7585,-63.8540,888a05915dfffff,0.94,...,Yes,Estadual,Urban,Yes,5.0,YES,4G,YES,YES,YES
1,115d27ba-82a1-3744-a3fb-90f216f54f6f,Brazil,government,G_BRA_000002,11000040,EMEIEF PEQUENOS TALENTOS,-8.7937,-63.8839,888a059319fffff,1.00,...,No,Municipal,Urban,Yes,5.0,YES,4G,YES,YES,YES


In [40]:
master_w_coverage_connectivity[master_w_coverage_connectivity['coverage_type']=='no coverage'].head(10)

,uuid,country,source,giga_school_id,school_id,name,lat,lon,hex8,dq_score,...,computer_availability,school_type,school_region,connectivity,internet_speed_mbps,coverage,coverage_type,2G_coverage,3G_coverage,4G_coverage
10,dd0e6466-2414-3281-9971-ca023cdd7b5a,Brazil,government,G_BRA_000011,11000449,EMEF ANIBAL MARTINS,-8.3259,-63.4920,888a05cdd3fffff,0.94,...,NaN,Municipal,Rural,NO,NaN,NO,no coverage,NO,NO,NO
16,79a70405-cccb-36b0-8c76-973466dda1e4,Brazil,government,G_BRA_000017,11000562,EMEF BARAO DO RIO BRANCO,-9.5285,-65.4588,888a2cc287fffff,1.00,...,No,Municipal,Urban,NO,0.0,NO,no coverage,NO,NO,NO
17,f4c0773c-e63d-32ab-af6b-f6db657c8ef6,Brazil,government,G_BRA_000018,11000589,EMEIF SENADOR DARCY RIBEIR0,-11.2232,-62.5241,888b1b26d7fffff,1.00,...,No,Municipal,Rural,NO,5.0,NO,no coverage,NO,NO,NO
19,46aae809-1dab-3c26-b8e7-2fc9c470839f,Brazil,government,G_BRA_000020,11000635,EMEF BOHEMUNDO ALVARES AFONSO,-9.0390,-63.8952,888a2339c9fffff,0.61,...,No,Municipal,Rural,NO,0.0,NO,no coverage,NO,NO,NO
24,3ea5a628-1407-3b2c-bd3d-da78e6098afe,Brazil,government,G_BRA_000025,11000724,EMEF CARANA,-9.0390,-63.8952,888a2339c9fffff,0.94,...,NaN,Municipal,Rural,NO,NaN,NO,no coverage,NO,NO,NO
27,176a418f-3441-3934-92cf-3b91f904f0a0,Brazil,government,G_BRA_000028,11000864,EMEF DEIGMAR MORAES DE SOUZA,-9.0390,-63.8952,888a2339c9fffff,0.61,...,No,Municipal,Rural,NO,5.0,NO,no coverage,NO,NO,NO
28,e24ed586-64f8-3441-a902-c833dfdcd22e,Brazil,government,G_BRA_000029,11000899,EMEF DR JOAO FERNANDES,-8.3259,-63.4920,888a05cdd3fffff,0.94,...,NaN,Municipal,Rural,NO,NaN,NO,no coverage,NO,NO,NO
30,da064fec-a575-3f42-af66-072a79bd28c2,Brazil,government,G_BRA_000031,11000945,EMEF DR RENATO MEDEIROS,-8.3259,-63.4920,888a05cdd3fffff,0.94,...,NaN,Municipal,Rural,NO,NaN,NO,no coverage,NO,NO,NO
34,534ff8f4-8ca5-3a19-935e-353cecb00279,Brazil,government,G_BRA_000035,11001011,EMEF ERMELINDO MONTEIRO BRASIL,-9.0390,-63.8952,888a2339c9fffff,0.61,...,No,Municipal,Rural,NO,5.0,NO,no coverage,NO,NO,NO
37,b02b9028-2023-38b0-962b-c9aa9a049d5d,Brazil,government,G_BRA_000038,11001070,EMEF FRANCISCO BRAGA,-8.3259,-63.4920,888a05cdd3fffff,0.61,...,No,Municipal,Rural,NO,0.0,NO,no coverage,NO,NO,NO


In [41]:
master_w_coverage_connectivity.head()

,uuid,country,source,giga_school_id,school_id,name,lat,lon,hex8,dq_score,...,computer_availability,school_type,school_region,connectivity,internet_speed_mbps,coverage,coverage_type,2G_coverage,3G_coverage,4G_coverage
0,82679ff0-f358-33d0-b3ce-4fc4c2740b7e,Brazil,government,G_BRA_000001,11000023,EEEE ABNAEL MACHADO DE LIMA - CENE,-8.7585,-63.8540,888a05915dfffff,0.94,...,Yes,Estadual,Urban,Yes,5.0,YES,4G,YES,YES,YES
1,115d27ba-82a1-3744-a3fb-90f216f54f6f,Brazil,government,G_BRA_000002,11000040,EMEIEF PEQUENOS TALENTOS,-8.7937,-63.8839,888a059319fffff,1.00,...,No,Municipal,Urban,Yes,5.0,YES,4G,YES,YES,YES
2,3d7fb661-61a4-33b3-bc3a-33455b75f7d9,Brazil,government,G_BRA_000003,11000201,EMEF PROF HERBERT DE ALENCAR,-8.7640,-63.8783,888a059061fffff,0.94,...,Yes,Municipal,Urban,Yes,5.0,YES,4G,YES,YES,YES
3,9507b9e6-4ea6-3cfc-b0c5-4c0b9f64c711,Brazil,government,G_BRA_000004,11000260,COLEGIO TIRADENTES DA POLICIA MILITAR DO ESTAD...,-8.7383,-63.8791,888a059021fffff,1.00,...,Yes,Estadual,Urban,Yes,5.0,YES,4G,YES,YES,YES
4,b5ec97ef-a418-35da-b161-3971c87ac36d,Brazil,government,G_BRA_000005,11000309,ESCOLA MUNICIPAL DE EDUCACAO INFANTIL MARISE C...,-8.7508,-63.9087,888a05900bfffff,1.00,...,No,Municipal,Urban,No,0.0,YES,4G,YES,YES,YES


In [42]:
master_w_coverage_connectivity['coverage'].value_counts()

YES    151816
NO      23888
Name: coverage, dtype: int64